# Analysis

In [1]:
import pandas as pd
import requests
import altair as alt

alt.renderers.enable('default')

%run ../altair-theme/arics_theme.py

In [2]:
%%html
<style>
@import url('https://fonts.googleapis.com');
@import url('https://fonts.gstatic.com');
@import url('https://fonts.googleapis.com/css2?family=Roboto+Mono&display=swap');
</style>

In [3]:
# Register the custom theme and give it a name.
alt.themes.register('arics_theme', arics_theme)

# Enable the theme
alt.themes.enable('arics_theme')

ThemeRegistry.enable('arics_theme')

In [4]:
contribs_df = pd.read_csv("mayoral_contribs.csv", usecols=[
    "recipientName", 
    "recipientCommitteeName", 
    "officeSought", 
    "ballotIssue", 
    "contributorName", 
    "address1", 
    "contributorCity", 
    "contributorStateCode", 
    "zipCode", 
    "amount", 
    "date"
], dtype={"zipCode": str}
)
contribs_df.head()

,recipientName,recipientCommitteeName,officeSought,ballotIssue,contributorName,amount,date,contributorCity,contributorStateCode,zipCode,address1
0,Ean Tafoya,Tafoya for Mayor,Mayor,NaN,Nicole Servino,25.0,2023-02-28T07:00:00,Denver,CO,80205,2947 Stout St
1,Ean Tafoya,Tafoya for Mayor,Mayor,NaN,Paula Bard,10.0,2023-02-28T07:00:00,Morrison,CO,80465,10250 city view drive
2,Ean Tafoya,Tafoya for Mayor,Mayor,NaN,Lauren Swain,5.0,2023-02-28T07:00:00,Denver,CO,80212,3277 N Raleigh St
3,Leslie Herod,Leslie for Mayor,Mayor,NaN,Tegan Kissane,50.0,2023-02-28T07:00:00,Denver,CO,80207,2580 Cherry Street
4,Leslie Herod,Leslie for Mayor,Mayor,NaN,Taylor Jordan,80.0,2023-02-28T07:00:00,Denver,CO,80205,2134 N Williams St


### Remove contributions to the current mayor.

In [5]:
contribs_df = contribs_df[~(contribs_df["recipientName"] == "Michael Hancock")]

In [6]:
# Total cash in the mayoral race
contribs_df["amount"].sum()

7276123.321

## Totals by recipient

In [7]:
# Clean up names before grouping
contribs_df["recipientName"] = contribs_df["recipientName"].str.replace("  ", " ")
contribs_df["recipientName"] = contribs_df["recipientName"].str.strip()
contribs_df["recipientName"] = contribs_df["recipientName"].str.replace("Calderón", "Caldeeeeron")

# Group and save
recipient_totals = contribs_df.groupby("recipientName")["amount"].sum().reset_index()
recipient_totals.to_csv("recipient_totals.csv", index=False)
recipient_totals.sort_values("amount", ascending=False)

,recipientName,amount
18,Kelly Brough,1148363.651
2,Andre Rougeot,797857.890
13,Jamie Giellis,783711.560
21,Leslie Herod,756602.030
25,Mike Johnston,728240.440
6,Chris Hansen,443011.510
17,Kayvan Khalatbari,389271.710
26,Penfield Tate,362870.810
7,Deborah Ortega,332727.090
19,Kwame Spearman,219074.250


In [8]:
recipient_totals["last_name"] = recipient_totals["recipientName"].apply(lambda x: x.split(" ")[-1])

bars = alt.Chart(recipient_totals).mark_bar().encode(
    x=alt.X('amount', axis=alt.Axis(format='$.2s', title='Contributions')),
    y=alt.Y('last_name', axis=alt.Axis(title=''), sort='-x'),
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text( 'amount:Q', format='$.3s'),
)


(bars + text).properties(
    title={
        "text":"Denver Mayoral candidate contributions",
#         "subtitle":"Cash raised as of March 12, 2023"
    },
    height=900,
    width=800
)

/Users/achokey/Library/Caches/pypoetry/virtualenvs/notebooks-loWCrMEE-py3.9/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

## Top donors

In [9]:
donor_totals = contribs_df[~contribs_df["contributorName"].isin(["Denver Fair Elections Fund Disbursement", "Previous Balance", "Unitemized"])]
donor_totals = donor_totals.groupby(["contributorName"])["amount"].sum().reset_index()
donor_totals_top_10 = donor_totals.sort_values("amount", ascending=False).reset_index().head(10)

In [10]:
donor_totals_top_10_chart = alt.Chart(donor_totals_top_10).mark_bar().encode(
    x=alt.X('amount', axis=alt.Axis(format='$.2s', title='Contributions')),
    y=alt.Y('contributorName', axis=alt.Axis(title='', labelFontSize=12), sort='-x', ),
)

donor_totals_top_10_text = donor_totals_top_10_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3 
).encode(
    text=alt.Text( 'amount:Q', format='$.3s'),
)


(donor_totals_top_10_chart + donor_totals_top_10_text).properties(
    title={
        "text":"Top donors",
#         "subtitle":"Cash raised as of March 12, 2023"
    },
    height=500,
    width=600
)

['Andre', 'Rougcot']
['Jamie', 'Giellis']
['Kayvan', 'Khalatbari']
['Penfield', 'III']
['Carol', 'Boigon']
['Zeppelin', 'Development']
['Roy', 'Romer']


/Users/achokey/Library/Caches/pypoetry/virtualenvs/notebooks-loWCrMEE-py3.9/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

## Top donors for each candidate

In [11]:
contributors_grouped = contribs_df.groupby(["recipientName", "contributorName"])["amount"].sum().reset_index()
contributors_grouped.to_csv("top_donors.csv", index=False)
for group_name, df_group in contributors_grouped.groupby(["recipientName"]):
    top_donors = df_group.sort_values("amount", ascending=False).reset_index().head(5)
    print(f"{group_name} top donors")
    print(top_donors)

Al Gardner top donors
   index recipientName     contributorName  amount
0     31    Al Gardner     Eddie Victorian   500.0
1     67    Al Gardner          Mark Jones   500.0
2     18    Al Gardner  Christopher Marino   500.0
3     16    Al Gardner       Chris Roberts   500.0
4     90    Al Gardner      Ronald Zappolo   500.0
Alex Valdez top donors
   index recipientName  contributorName  amount
0    292   Alex Valdez      John Morris  1000.0
1    416   Alex Valdez  Philip Kingston   500.0
2    488   Alex Valdez     Travis Berry   500.0
3    149   Alex Valdez    Beth Kerttula   500.0
4    151   Alex Valdez     Bob Satawake   500.0
Andre Rougeot top donors
   index  recipientName                                    contributorName  \
0    517  Andre Rougeot                                      Andre Rougcot   
1    520  Andre Rougeot  Apartment Association of Metro Denver Denver S...   
2    587  Andre Rougeot                                       Kaylyn Stone   
3    509  Andre Rougeot 

/var/folders/bq/b_2ct9jx66z1nh9qsrcwrbhh0000gn/T/ipykernel_24852/3049385735.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group_name, df_group in contributors_grouped.groupby(["recipientName"]):


## Out of state donations

In [12]:
# What's the total from out of state?
outside_co_df = contribs_df[contribs_df["contributorStateCode"].str.upper() != "CO"]
oos_total = outside_co_df["amount"].sum()

In [13]:
outside_co_df.groupby(["contributorName", "contributorStateCode"])["amount"].sum().reset_index()

,contributorName,contributorStateCode,amount
0,Anthony Spikes,NY,500.00
1,Ariel Kiyomi Daoud,OH,50.00
2,Asa Tate,WA,100.00
3,Audrey Kennedy,CA,250.00
4,Christina Northern,DC,150.00
...,...,...,...
2482,stuart pierre,NY,1.25
2483,tabitha creel,AZ,100.00
2484,thomas england,NY,6.25
2485,todd stribley,VA,50.00


In [14]:
outside_co_df["contributorStateCode"] = outside_co_df["contributorStateCode"].str.upper()
outside_by_state = outside_co_df.groupby(["contributorStateCode"])["amount"].sum().reset_index()


# Chart
outside_by_state_chart = alt.Chart(outside_by_state).mark_bar().encode(
    x=alt.X('amount', axis=alt.Axis(format='$.2s', title='Contributions')),
    y=alt.Y('contributorStateCode', axis=alt.Axis(title=''), sort='-x'),
)

outside_by_state_text = outside_by_state_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text( 'amount:Q', format='$.3s'),
)


(outside_by_state_chart + outside_by_state_text).properties(
    title={
        "text":"Out-of-state sources",
#         "subtitle":"Cash raised as of March 12, 2023"
    },
    height=1200,
    width=800
)

/var/folders/bq/b_2ct9jx66z1nh9qsrcwrbhh0000gn/T/ipykernel_24852/840519579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outside_co_df["contributorStateCode"] = outside_co_df["contributorStateCode"].str.upper()
/Users/achokey/Library/Caches/pypoetry/virtualenvs/notebooks-loWCrMEE-py3.9/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

## Top out-of-state donors

In [28]:
top_outside_donors = outside_co_df.groupby(["contributorName", "contributorStateCode"])["amount"].sum().reset_index()
top_outside_donors = top_outside_donors.sort_values("amount", ascending=False).head(10)

In [31]:
# Chart
top_outside_donors_chart = alt.Chart(top_outside_donors).mark_bar().encode(
    x=alt.X('amount', axis=alt.Axis(format='$.2s', title='Contributions')),
    y=alt.Y('contributorName', axis=alt.Axis(title=''), sort='-x'),
)

top_outside_donors_text = top_outside_donors_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text( 'amount:Q', format='$.2s'),
)


(top_outside_donors_chart + top_outside_donors_text).properties(
    title={
        "text":"Top out-of-state donors",
#         "subtitle":"Cash raised as of March 12, 2023"
    },
    height=400,
    width=800
)

alt.LayerChart(...)